In [20]:
import os
import re
from ast import literal_eval
from dotenv import load_dotenv
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    wait_random
)  # for exponential backoff

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

COMPLETIONS_MODEL = "gpt-3.5-turbo"
# COMPLETIONS_MODEL = "gpt-4"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [3]:
df = pd.read_csv('temple_details_v33.csv')
df['state'] = df['state'].apply(lambda x: str(x).replace(',','_').replace(' ', '_'))
df['temple_name'] = df['temple_name'].apply(lambda x: str(x).replace(',','_').replace(' ', '_'))
df = df.drop_duplicates().reset_index(drop=True)

In [5]:
# df = df.loc[df['state']=='Karnataka']

In [6]:
df = df.set_index(['state', 'temple_name', 'sent_grp'])

In [7]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(10))
# @retry(wait=wait_random(min=60, max=120), stop=stop_after_attempt(10))
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    text = re.sub(r'[^\w_. -]', ' ', text).replace('  ', ' ')
    text = text.strip().encode('latin-1', 'replace').decode('latin-1')
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.temple_details) for idx, r in df.iterrows()
    }

In [8]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "state" and c != "temple_name" and c != "sent_grp"])
    return {
           (r.state, r.temple_name, r.sent_grp): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [9]:
document_embeddings = load_embeddings("temple_details_embeddings_v33.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings = compute_doc_embeddings(df)
# document_embeddings_df = pd.DataFrame(document_embeddings.items())
# df2 = document_embeddings_df[0].apply(pd.Series)
# df2.columns=['state', 'temple_name', 'sent_grp']

# df3 = document_embeddings_df[1].apply(pd.Series)

# df2 = pd.concat([df2, df3], axis=1)
# df2.to_csv('ka_temple_details_embeddings_v33.csv', index=False)

In [10]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('Andaman_and_Nicobar_Islands', 'Radhakrishna_Temple__Port_Blair', 0) : [-0.0094528701156377, -0.0167095530778169, 0.0031404830515384, 0.0159818995743989, -0.0216575916856527]... (1536 entries)


In [11]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [12]:
order_document_sections_by_query_similarity("List all the temples in Sringeri", document_embeddings)[:5]

[(0.860318071772364, ('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 0)),
 (0.8517578727446681,
  ('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)),
 (0.848474832096964, ('Maharashtra', 'Vigneshwara_Temple__Pune', 1)),
 (0.8464409085203286,
  ('Tamil_Nadu', 'Parthasarathy_Temple__Triplicane__Chennai', 7)),
 (0.8461109553508552, ('Karnataka', 'Siddalingeshwar_Temple__Yadagiri', 0))]

In [32]:
MAX_SECTION_LEN = 1500
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [30]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)[:3]
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        print(section_index)
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index].iloc[0]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + 'State: ' + section_index[0] + ', Temple_City: ' + section_index[1] + ', Temple Details: ' +
        document_section['temple_details'].replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    # print(f"Selected {len(chosen_sections)} document sections:")
    # print("\n".join(chosen_sections_indexes))
    
    # header = """Answer the question as truthfully as possible using the provided context, and if the answer is not specified within the text below, say "I don't know."\n\nContext:\n"""
    header = """Answer truthfully based on context.\n\nIf users ask for a list, try finding relevant contexts and create a list from them.\n\nContext:\n"""
    # header = """\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [209]:
prompt = construct_prompt(
    "List all the temples in Sringeri",
    document_embeddings,
    df
)

print("===\n", prompt)

('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 0)
('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Maharashtra', 'Vigneshwara_Temple__Pune', 1)
('Tamil_Nadu', 'Parthasarathy_Temple__Triplicane__Chennai', 7)
('Karnataka', 'Siddalingeshwar_Temple__Yadagiri', 0)
('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Odisha', 'Swarnadhisvara_Siva_Temple__Bhubaneswar__Odisha', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('Tamil_Nadu', 'Sangameswarar_Temple__Erode__Tamil_Nadu', 1)
('Tamil_Nadu', 'Jambukeswarar_Temple__Tiruchirappalli__Tamil_Nadu', 1)
('Kerala', 'Ananthapura_Lake_Temple__Kasaragod__Kerala', 1)
('Tamil_Nadu', 'Ekambareswarar_Temple__Kanchipuram__Tamil_Nadu', 1)
('Tamil_Nadu', 'Sampangi_Pitchaaleeswarar_Temple__Arani__Thiruvallur', 0)
===
 Answer truthfully based on context.

If users ask for a list, try finding relevant contexts and create

C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


In [15]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=[{'role': 'user', 'content': prompt}]
    )

    return response["choices"][0]["message"]["content"].strip(" \n")

In [211]:
answer_query_with_context("List all the temples in Sringeri", df, document_embeddings)

('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 0)
('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Maharashtra', 'Vigneshwara_Temple__Pune', 1)
('Tamil_Nadu', 'Parthasarathy_Temple__Triplicane__Chennai', 7)
('Karnataka', 'Siddalingeshwar_Temple__Yadagiri', 0)
('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Odisha', 'Swarnadhisvara_Siva_Temple__Bhubaneswar__Odisha', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('Tamil_Nadu', 'Sangameswarar_Temple__Erode__Tamil_Nadu', 1)
('Tamil_Nadu', 'Jambukeswarar_Temple__Tiruchirappalli__Tamil_Nadu', 1)
('Kerala', 'Ananthapura_Lake_Temple__Kasaragod__Kerala', 1)
('Tamil_Nadu', 'Ekambareswarar_Temple__Kanchipuram__Tamil_Nadu', 1)
('Tamil_Nadu', 'Sampangi_Pitchaaleeswarar_Temple__Arani__Thiruvallur', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'- Sringeri Sharada Peetham'

In [212]:
answer_query_with_context("What is different between kukke and ghati subrahmanya temples?", df, document_embeddings)

('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 2)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 0)
('Kerala', 'Vaikom_Temple__Kottayam__Kerala', 1)
('Kerala', 'Thirunavaya_Navamukunda_Temple__Malappuram__kerala', 3)
('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 0)
('Uttarakhand', 'Guptakashi__Rudraprayag__Uttrakhand', 2)
('Karnataka', 'Kodlamane__Uttara_kannada__Karnataka', 1)
('Kerala', 'Chottanikkara_Temple__Ernakulam__Kerala', 1)
('Gujarat', 'Sun_Temple__Modhera__Mehsana__Gujarat', 1)
('Kerala', 'Kottiyoor_Temple__kannur__Kerala', 2)
('Odisha', 'Jaleswar_Siva_Temple_Precinct___Bhubaneswar__Odisha', 3)
('Tamil_Nadu', 'Kandhaswamy_Temple__Cheyyur__Kanchipuram', 5)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The difference between Kukke Subramanya Temple and Ghati Subramanya Temple is that Kukke Subramanya Temple is located in the village of Subramanya and worships Kartikeya as Subramanya, lord of all serpents, while Ghati Subramanya Temple is located on the outskirts of Bangalore near Tubagere, Doddaballapura and is one of the major spots for snake worship in Southern India, with the prime deity Lord Karthikeya found together with Lord Narasimha.'

In [213]:
answer_query_with_context("What are the hill top temples in karnataka?", df, document_embeddings)

('Karnataka', 'Devarayanadurga__Tumkuru__karnatka', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 3)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 4)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)
('Karnataka', 'Halebidu__Hassan__Karnataka', 2)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 6)
('Karnataka', 'Galageshwara_Temple__Haveri', 0)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'Bhoganarasimha, Yoganarasimha and Laxmi Narasimha at Devarayanadurga in Tumkuru, and Chamundeshwari Temple at Chamundi Hills in Mysore are the hill top temples in Karnataka mentioned in the given context.'

In [129]:
answer_query_with_context("What is the oldest temple in bengaluru?", df, document_embeddings)

('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Halebidu__Hassan__Karnataka', 2)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The oldest temple in Bengaluru is Sri Dharmaraya Swamy temple. It is believed to be about 800 years old and was built by the Ganga Arasu people.'

In [130]:
answer_query_with_context("What are the other old temples in bengaluru?", df, document_embeddings)

('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Hoysaleshwara_Temple__Hassan', 0)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)
('Karnataka', 'Halebidu__Hassan__Karnataka', 2)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 6)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The Sri Dharmaraya Swamy temple and Gavi Gangadhareshwara Temple are two old temples in Bengaluru mentioned in the given context.'

In [131]:
answer_query_with_context("what is the document about?", df, document_embeddings)

('Karnataka', 'Padutirupathi__Udupi__Karnataka', 5)
('Karnataka', 'Dharmasthala_Temple__Dakshina_Kannada__Karnataka', 1)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 5)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 6)
('Karnataka', 'Annapoorneshwari_Temple__Horanadu__Karnataka', 3)
('Karnataka', 'Murdeshwar__Uttara_Kannada__Karnataka', 0)
('Karnataka', 'Aihole__Bagalkot__Karnataka', 2)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 4)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 1)
('Karnataka', 'Badami__Bagalkot__Karnataka', 3)
('Karnataka', 'Kodlamane__Uttara_kannada__Karnataka', 0)
('Karnataka', 'Papnash_Shiva_Temple__Bidar', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The document provides information about various temples in the state of Karnataka, their locations, history, and details about the deities and rituals associated with them.'

In [132]:
answer_query_with_context("list all the temples near mysore", df, document_embeddings)

('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 4)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 5)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 5)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 7)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 6)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 1)
('Karnataka', 'Tarakeshwara_Temple__Haveri', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 0)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 4)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Chamundeshwari Temple, Mysore\n2. Cheluvanarayana Swamy Temple, Mandya\n3. Jwalamuki Shri Tripurasundari Temple, Utthanahalli (near Chamundeshwari Temple, Mysore)\n4. Dharmaraya Swamy Temple, Bengaluru (not near Mysore, but in the state of Karnataka)\n5. Amrutesvara Temple, Amruthapura, Chikkamagaluru (not near Mysore, but in the state of Karnataka)\n6. Tarakeshwara Temple, Haveri (not near Mysore, but in the state of Karnataka)'

In [137]:
answer_query_with_context("list all the temples near kukke subrahmanya", df, document_embeddings)

('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 0)
('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 1)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 5)
('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 2)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 4)
('Karnataka', 'Tarakeshwara_Temple__Haveri', 0)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 1)
('Karnataka', 'Kodlamane__Uttara_kannada__Karnataka', 3)
('Karnataka', 'Devarayanadurga__Tumkuru__karnatka', 0)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 2)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'There is no specific list of temples near Kukke Subrahmanya provided in the context. The context provides details about Kukke Subrahmanya Temple itself and its significance.'

In [138]:
answer_query_with_context("what is the history of sringeri temple? who did tapas there?", df, document_embeddings)

('Karnataka', 'Padutirupathi__Udupi__Karnataka', 2)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 2)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 1)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 4)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 3)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 2)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 3)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The history of Sringeri temple dates back more than twelve centuries ago when Adi Shankaracharya established the first of the four Amnaya Peethams. The Acharya chose Sringeri as the place to stay and teach his disciples after seeing a cobra providing shelter from the hot sun to a frog undergoing labor. Rishyashringa is also said to have performed tapas at Sringeri. The temple is known for being a centre of spiritual power and a great place of traditional learning owing to the presence of Goddess Sharada and the erudition of the Acharyas of the Peetham.'

In [139]:
answer_query_with_context("what is the history of chamundeswari temple? how to reach the temple from mysore bus stand?", df, document_embeddings)

('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 3)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 4)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 5)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 6)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 2)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Tarakeshwara_Temple__Haveri', 0)
('Karnataka', 'Anegudde__Udupi__Karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Siddhesvara_temple__Haveri', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 5)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The Chamundeshwari Temple is a Hindu temple located on top of Chamundi Hills, about 13 km from the palace city of Mysore in the state of Karnataka, India. It is a temple of great antiquity with over 1,000 years of history, initially built as a small shrine and eventually assumed religious prominence after the Mysuru Maharajas came to power in 1399 A.D. It is named after Chamundeshwari or Durga, the fierce form of Shakti and believed to be an incarnation of Parvati. To reach the temple from Mysore bus stand, there is a good motorable road to the top of the hills and city bus facilities are available. Karnataka State Road Transport Corporation (KSRTC) also operates regular bus services every day.'

In [140]:
answer_query_with_context("what is the history of the oldest shiva temple in bengaluru?", df, document_embeddings)

('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Halebidu__Hassan__Karnataka', 2)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 3)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


"The Gavi Gangadhareshwara temple in Bengaluru is one of the oldest temples in the city, built in the 16th century AD by Kempe Gowda, the founder of Bengaluru. It is a cave temple dedicated to Shiva and has a tall Shivalinga in the inner sanctum. One of the major attractions of the temple is a rare idol of Agni, the God of fire, with two heads, seven hands, and three legs. It is perhaps the only temple in South India that has such an idol. The temple also has a wonderful sculpture of Nandi, Shiva's mount, and a beautiful image of Shakti Ganapathi with 12 hands at the left side of the main entrance. Those who worship the deity Agnimurthi believe it will cure one of the defects of the eye."

In [141]:
answer_query_with_context("are there other agni temples in karnataka?", df, document_embeddings)

('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 5)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 4)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Galageshwara_Temple__Haveri', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Kotilingeshwar_Temple__Kolar', 0)
('Karnataka', 'Kodlamane__Uttara_kannada__Karnataka', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The context only mentions the presence of a rare idol of Agni, the God of fire, in Gavi Gangadhareshwara Temple, Bengaluru. The information provided does not specify if there are other Agni temples in Karnataka.'

In [152]:
answer_query_with_context("combine all the contexts and list the krishna temples in karnataka", df, document_embeddings)

('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 5)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 1)
('Karnataka', 'ISKCON_Temple__Bangalore__karnataka', 0)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 6)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 0)
('Karnataka', 'Udupi_Sri_Krishna_Matha__Udupi__karnataka', 0)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 4)
('Karnataka', 'Tarakeshwara_Temple__Haveri', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\2026104477.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Udupi Sri Krishna Matha, Udupi\n2. Padutirupathi, Udupi\n3. ISKCON Temple, Bangalore\n4. Chennakesava Temple, Belur, Hassan\n5. Sri Dharmaraya Swamy Temple, Bangalore \n6. Amrutesvara Temple, Amruthapura, Chikkamagaluru\n7. Tarakeshwara Temple, Haveri'

In [180]:
answer_query_with_context("list the temples in mysore", df, document_embeddings)

('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 4)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 5)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 7)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 0)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 4)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 0)
('Karnataka', 'Chamundeshwari_Temple__Mysore__karnataka', 1)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 6)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Chamundeshwari Temple, Mysore\n2. Mahabaleswara Temple, Mysore (located atop Chamundi Hills)\n3. Jwalamuki Shri Tripurasundari Temple (located at the foothill of Chamundi Hills)'

In [161]:
answer_query_with_context("what are the temples in sringeri?", df, document_embeddings)

('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 3)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 4)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 5)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 2)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 2)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Siddalingeshwar_Temple__Yadagiri', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The main temple in Sringeri is Sri Kshetra Shringeri, which is the site of the first matha established by Adi Shankara in the 8th century CE. The temple is dedicated to Goddess Sharada and is also known as the Dakshinamnaya Sri Sharada Peetham.'

In [197]:
answer_query_with_context("which temples did shankaracharya visit?", df, document_embeddings)

('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 1)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 4)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 2)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 3)
('Karnataka', 'Cheluvanarayana_Swamy_Temple__Mandya__Karnataka', 5)
('Karnataka', 'Sringeri__Chikkamagaluru__karnataka', 1)
('Karnataka', 'Kukke_Subramanya_Temple__Dakshina_Kannada__karnataka', 2)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 2)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)
('Karnataka', 'Udupi_Sri_Krishna_Matha__Udupi__karnataka', 1)
('Karnataka', 'Annapoorneshwari_Temple__Horanadu__Karnataka', 3)
('Karnataka', 'Dharmasthala_Temple__Dakshina_Kannada__Karnataka', 1)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The context mentions that Shankaracharya camped at Kukke Subramanya Temple in Dakshina Kannada and also established the Sharada Peetham at Sringeri in South India.'

In [169]:
answer_query_with_context("what is special about karnataka's temples?", df, document_embeddings)

('Karnataka', 'Padutirupathi__Udupi__Karnataka', 1)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 2)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 4)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 0)
('Karnataka', 'Kodlamane__Uttara_kannada__Karnataka', 0)
('Karnataka', 'Nellitheertha_Cave_Temple__Dakshina_Kannada__karnataka', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1047947636.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


"Karnataka's temples are known for their unique architecture and historical significance. They also have various deities worshipped and special rituals performed during festivals. Many of these temples have rare idols and sculptures that are not found anywhere else in South India. Additionally, some of these temples also focus on social and academic causes, offering support and aid to various institutions and organizations that benefit people of both the temple cities and beyond."

In [186]:
answer_query_with_context("list all temples which are more than 500 years old", df, document_embeddings)

('Karnataka', 'Badami_cave_temples__Badami__Karnataka', 3)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)
('Karnataka', 'Aihole__Bagalkot__Karnataka', 1)
('Karnataka', 'Badami_cave_temples__Badami__Karnataka', 2)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 4)
('Karnataka', 'Badami_cave_temples__Badami__Karnataka', 4)
('Karnataka', 'Ghati_Subramanya__Bangalore__Karnataka', 2)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 2)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 2)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Badami Cave Temples in Badami, Karnataka \n2. Chennakeshava Temple in Belur, Karnataka \n3. Shree Vishnumurthy Temple in Mangalore, Karnataka \n4. Hoysaleswara Temple in Hassan, Karnataka \n5. Amrutesvara Temple in Amruthapura, Chikkamagaluru, Karnataka \n6. Ghati Subramanya Temple in Bangalore, Karnataka'

In [196]:
answer_query_with_context("was iskcon bangalore built in 20th century?", df, document_embeddings)

('Karnataka', 'ISKCON_Temple__Bangalore__karnataka', 0)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'ISKCON_Temple__Bangalore__karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 3)
('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 6)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 2)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 4)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Mahadeva_Temple__Koppal', 0)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 2)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'No, the ISKCON temple in Bangalore was not built in the 20th century. The society applied for land in 1987 and the temple was constructed on the allotted land in 1997.'

In [195]:
answer_query_with_context("list temples in bengaluru built after 1950", df, document_embeddings)

('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 0)
('Karnataka', 'Gavi_Gangadhareshwara_Temple__Bengaluru__Karnataka', 0)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Karnataka', 'Halebidu__Hassan__Karnataka', 2)
('Karnataka', 'Chennakeshava_Temple__Belur__Hassan__Karnataka', 1)
('Karnataka', 'Dharmaraya_Swamy_Temple__Â\xa0Bengaluru__Karnataka', 2)
('Karnataka', 'Marikamba_Temple__Sirsi__Karnataka', 0)
('Karnataka', 'Dodda_Basavana_Gudi__Bengaluru__Karnataka', 1)
('Karnataka', 'Hoysaleswara_Temple__Hassan__karnataka', 3)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Karnataka', 'Virupaksha_Temple__Hampi__Bellary__Karnataka', 3)
('Karnataka', 'Mahadeva_Temple__Koppal', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'There is no information in the given context about temples in Bengaluru built after 1950.'

In [214]:
answer_query_with_context("What are the hill top temples in andhra pradesh?", df, document_embeddings)

('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)
('Andhra_Pradesh', 'Veerabhadra_Temple__Lepakshi__Andhara_Pradesh', 0)
('Andhra_Pradesh', 'Kapila_Theertham__Andhra_Pradesh', 0)
('Andhra_Pradesh', 'Venkateswara_Temple__Tirumala__Andhra_Pradesh', 0)
('Andhra_Pradesh', 'Peddintlamma_Temple__Krishna', 0)
('nan', 'Kuchadri_Venkateshwara_Swamy_temple__Medak__Telangana', 1)
('Tamil_Nadu', 'Thiruneermalai_Dhivya_Desam__Kanchipuram', 3)
('nan', 'Thousand_Pillar_Temple__Hanamakonda__Telangana', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('Bihar', 'Vishnupad_Mandir__Bihar', 1)
('nan', 'Kuchadri_Venkateshwara_Swamy_temple__Medak__Telangana', 2)
('nan', 'Bhadrakali_Temple__Warangal__Telangana', 0)
('Andhra_Pradesh', 'Mallikarjuna__Jyotirlinga__Andhra_Pradesh', 0)
('Karnataka', 'Badami_cave_temples__Badami__Karnataka', 2)
('nan', 'Kuchadri_Venkateshwara_Swamy_temple__Medak__Telangana', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'There are several hill top temples in Andhra Pradesh, including Veerabhadra Temple in Lepakshi, Kapila Theertham in Tirupati, Venkateswara Temple in Tirumala, and Mallikarjuna Jyotirlinga in Srisailam.'

In [215]:
answer_query_with_context("What are the hill top temples in telangana?", df, document_embeddings)

('nan', 'Kuchadri_Venkateshwara_Swamy_temple__Medak__Telangana', 1)
('nan', 'Thousand_Pillar_Temple__Hanamakonda__Telangana', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('nan', 'Bhadrakali_Temple__Warangal__Telangana', 0)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)
('nan', 'Bhadrakali_Temple__Warangal__Telangana', 4)
('nan', 'Chilkoor_Balaji_Temple__Hyderabad__Telagana', 1)
('nan', 'Bhadrakali_Temple__Warangal__Telangana', 3)
('Bihar', 'Vishnupad_Mandir__Bihar', 1)
('Andhra_Pradesh', 'Peddintlamma_Temple__Krishna', 0)
('nan', 'Kuchadri_Venkateshwara_Swamy_temple__Medak__Telangana', 0)
('Andhra_Pradesh', 'Venkateswara_Temple__Tirumala__Andhra_Pradesh', 0)
('nan', 'Keesaragutta_Temple__Medchal-Malkajgir__Telagana', 0)
('Andhra_Pradesh', 'Kapila_Theertham__Andhra_Pradesh', 0)
('nan', 'Padmakshi_Temple__Warangal__Telangana', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Kuchadri Venkateshwara Swamy temple, Medak.\n2. Thousand Pillar Temple, Hanamakonda.\n3. Bhadrakali Temple, Warangal.'

In [216]:
answer_query_with_context("What are the hill top temples in tamil nadu?", df, document_embeddings)

('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 0)
('Tamil_Nadu', 'Arupadai_Veedu_Temple_Thiruthani__palani', 4)
('Tamil_Nadu', 'Thalamalai_Perumal_Temple__Trichy', 2)
('Tamil_Nadu', 'Idumban_Hill_Temple__palani', 0)
('Tamil_Nadu', 'Arupadai_Veedu_Temple__Pazhamuthicholai__Madurai', 1)
('Tamil_Nadu', 'Arupadai_Veedu_Temple__Pazhamuthicholai__Madurai', 0)
('Tamil_Nadu', 'Jain_Cave_Temple_Complex__Yanaimalai__Madurai', 1)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 0)
('Tamil_Nadu', 'Dhandayuthapani_Murugan_Temple__Nadu_Palani__Kanchipuram', 0)
('Tamil_Nadu', 'Annamalaiyar_Temple__Tiruvannamalai', 0)
('Tamil_Nadu', 'Subramanya_Swamy_Temple__Kailasagiri__Gadambur__Vellore', 2)
('Tamil_Nadu', 'Thiruneermalai_Dhivya_Desam__Kanchipuram', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Subramanya Swamy Temple, Kailasagiri, Gadambur, Vellore\n2. Arupadai Veedu Temple, Thiruthani/Palani\n3. Thalamalai Perumal Temple, Trichy\n4. Idumban Hill Temple, Palani\n5. Arupadai Veedu Temple, Pazhamuthicholai, Madurai\n6. Jain Cave Temple Complex, Yanaimalai, Madurai\n7. Thirukkadigai Temple, Sholinghur, Vellore\n8. Dhandayuthapani Murugan Temple, Nadu Palani, Kanchipuram\n9. Annamalaiyar Temple, Tiruvannamalai\nNote: This list was created by compiling information from the provided context. Other hill top temples in Tamil Nadu may exist that are not included on this list.'

In [217]:
answer_query_with_context("what is the document about?", df, document_embeddings)

('Tamil_Nadu', 'Dharma_Shasta_Temple__Vettuveni__Marthandam__Kanyakumari', 1)
('Odisha', 'Dhabaleswar__Cuttack__Odisha', 4)
('Odisha', 'Nilkantheswar_Shiva_Temple__Bhubaneswar__Odisha', 4)
('Karnataka', 'Padutirupathi__Udupi__Karnataka', 5)
('Kerala', 'Thirunavaya_Navamukunda_Temple__Malappuram__kerala', 5)
('Andhra_Pradesh', 'Mallikarjuna__Jyotirlinga__Andhra_Pradesh', 1)
('Gujarat', 'Girnar__Junagadh__Gujarat', 5)
('Odisha', 'Sakshigopal_Temple__puri__Odisha', 1)
('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 1)
('Odisha', 'Nrusinghanath_Temple__Bargarh__Odisha', 1)
('Delhi', 'Yogmaya_Temple__New_Delhi', 5)
('Rajasthan', 'Nakkash_Ki_Devi_â\x80\x93_Gomti_Dham__Karauli__Rajasthan', 2)
('Uttar_Pradesh', 'Jwara_Hareshwar_Temple__Varanasi', 2)
('Rajasthan', 'Deo_somnath_temple__Dungarpur_(Rajasthan)', 1)
('Gujarat', 'Hanuman_temple__Sarangpur__Botad', 1)
('West_Bengal', 'Hangseshwari_Temple__Hooghly__West_Bengal', 5)
('Jharkhand', 'SUN_TEMPLE__Ranchi__Jharkhand', 0)
(

C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The document is about various temples'

In [218]:
answer_query_with_context("which temples have a path which devotees can circumambulate in?", df, document_embeddings)

('Karnataka', 'Chennakesava_Temple__Somanathapura__Mysore__Karnataka', 3)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)
('Karnataka', 'Udupi_Sri_Krishna_Matha__Udupi__karnataka', 1)
('Kerala', 'Padmanabhaswamy_Temple__Thiruvananthapuram__Kerala', 2)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 3)
('Kerala', 'Sabarimala__Pathanamthitta__Kerala', 3)
('Tamil_Nadu', 'Vamanapureeswarar_Temple__Thirumanikuzhi__Cuddalore', 0)
('Kerala', 'Padmanabhaswamy_Temple__Thiruvananthapuram__Kerala', 4)
('Tamil_Nadu', 'Navaneetha_Krishnan_Temple__Arasalur__Trichy', 1)
('Tamil_Nadu', 'Srivilliputhur_Andal_temple__Virudhunagar__Tamil_Nadu', 3)
('Uttar_Pradesh', 'Vishweshwar_Vishwanath_Temple__Varanasi', 6)
('Tamil_Nadu', 'Keelaparasalur_Veeratteswarar_Temple__Nagapattinam', 2)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Chennakesava Temple, Somanathapura, Mysore, Karnataka\n2. Thirukkadigai Temple, Sholinghur, Vellore, Tamil Nadu\n3. Udupi Sri Krishna Matha, Udupi, Karnataka \n4. Padmanabhaswamy Temple, Thiruvananthapuram, Kerala\n5. Srivilliputhur Andal Temple, Virudhunagar, Tamil Nadu'

In [219]:
answer_query_with_context("which tamil nadu temples have a path which devotees can circumambulate in?", df, document_embeddings)

('Tamil_Nadu', 'Swamithope_Pathi__Kanyakumari', 0)
('Tamil_Nadu', 'Vamanapureeswarar_Temple__Thirumanikuzhi__Cuddalore', 0)
('Tamil_Nadu', 'Dhandayuthapani_Murugan_Temple__Nadu_Palani__Kanchipuram', 0)
('Tamil_Nadu', 'Thirukkadigai_Temple__Sholinghur__Vellore', 9)
('Tamil_Nadu', 'Thanumalayan_Temple__Suchindram__Kanyakumari', 0)
('Tamil_Nadu', 'Kalamega_Perumal_Temple__Thirumohoor__Madurai', 0)
('Tamil_Nadu', 'Palvannanathar_Temple__Tirukkazhippalai__Chidambaram__Cuddalore', 0)
('Tamil_Nadu', 'Oondreeswarar_Temple__Poondi__Thiruvallur', 1)
('Tamil_Nadu', 'Eri_Katha_Ramar_Temple__Thiruninravur__Thiruvallur', 1)
('Tamil_Nadu', 'Navaneetha_Krishnan_Temple__Arasalur__Trichy', 1)
('Tamil_Nadu', 'Thirunandheeswarar_Temple__Manavur__Thiruvallur', 0)
('Tamil_Nadu', 'Tirupporur_Murugan_Temple__Kanchipuram', 2)
('Tamil_Nadu', 'Theerthapureeswarar_Temple__Thirunelvayil_Arathurai__Cuddalore', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'Swamithope Pathi, Kanyakumari; Vamanapureeswarar Temple, Thirumanikuzhi, Cuddalore; Dhandayuthapani Murugan Temple, Nadu Palani, Kanchipuram; Thirukkadigai Temple, Sholinghur, Vellore; Thanumalayan Temple, Suchindram, Kanyakumari; Kalamega Perumal Temple, Thirumohoor, Madurai; Palvannanathar Temple, Tirukkazhippalai, Chidambaram, Cuddalore; Navaneetha Krishnan Temple, Arasalur, Trichy; Thirunandheeswarar Temple, Manavur, Thiruvallur; Tirupporur Murugan Temple, Kanchipuram.'

In [220]:
answer_query_with_context("what are the famous temples in madurai?", df, document_embeddings)

('Tamil_Nadu', 'Meenakshi_Temple__Madurai__Tamil_Nadu', 4)
('Tamil_Nadu', 'Meenakshi_Temple__Madurai__Tamil_Nadu', 0)
('Tamil_Nadu', 'Kalamega_Perumal_Temple__Thirumohoor__Madurai', 0)
('Tamil_Nadu', 'Arupadai_Veedu_Temple__Pazhamuthicholai__Madurai', 1)
('Tamil_Nadu', 'Immayilum_Nanmai_Tharuvar_Temple__Madurai', 1)
('Tamil_Nadu', 'Edaganathar_Temple__Thiruvedagam__Madurai', 0)
('Tamil_Nadu', 'Theerthapureeswarar_Temple__Thirunelvayil_Arathurai__Cuddalore', 0)
('Tamil_Nadu', 'Veerapandi_Gowmariamman_temple__Theni', 0)
('Tamil_Nadu', 'Pasupatheeswarar_Temple__Kallar_Pasupathi_Koil__Thanjavur', 0)
('Tamil_Nadu', 'Dhandayuthapani_Murugan_Temple__Nadu_Palani__Kanchipuram', 1)
('Tamil_Nadu', 'Aadhi_Kesava_Perumal_Temple__Vada_Madurai__Thiruvallur', 2)
('Tamil_Nadu', 'Nellaiappar_Temple__Tirunelveli__Tamil_Nadu', 1)
('Tamil_Nadu', 'Thyagaraja_Temple__Tiruvarur__Tamil_Nadu', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Meenakshi Temple, Madurai, Tamil Nadu\n2. Kalamega Perumal Temple, Thirumohoor, Madurai\n3. Arupadai Veedu Temple, Pazhamuthicholai, Madurai\n4. Immayilum Nanmai Tharuvar Temple, Madurai\n5. Edaganathar Temple, Thiruvedagam, Madurai'

In [221]:
answer_query_with_context("what are the major shiva temples in andhra pradesh?", df, document_embeddings)

('Andhra_Pradesh', 'Yaganti_temple__Andhra_Pradesh', 1)
('Andhra_Pradesh', 'Pallikondeswarar_Temple__Surutapalli__Chittoor', 0)
('Odisha', 'Chateshwar_Temple__Cuttack__Odisha', 1)
('Tamil_Nadu', 'Ujjaini_Mahakali_Temple__Secunderabad__Telangana', 2)
('Tamil_Nadu', 'Brihadisvara_Temple__Thanjavur__Tamil_Nadu', 1)
('Andhra_Pradesh', 'Neelakanteshwara_Temple___East_Godavari', 0)
('Andhra_Pradesh', 'Ramalingeswara_Swamy_Temple__Prakasam', 0)
('Andhra_Pradesh', 'Maddi_Anjaneya_Temple__Godavari', 0)
('Tamil_Nadu', 'Vaitheeswaran_Koil__Nagapattinam__Tamil_Nadu', 4)
('Andhra_Pradesh', 'Kalahasteeswara_Swamy_Temple__Sri_Kalahasthi__Andhra_Pradesh', 0)
('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Tamil_Nadu', 'Bhadrachala_Ramar_Temple__Erumai_Vetti_Palayam__Thiruvallur', 1)
('Tamil_Nadu', 'Aadhi_Kesava_Perumal_Temple__Vada_Madurai__Thiruvallur', 1)
('Puducherry', 'Irumbai_Shiva_Temple__Pondicherry', 0)
('Andhra_Pradesh', 'Srikalahasti_Temple', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Yaganti Temple\n2. Pallikondeswarar Temple, Surutapalli, Chittoor\n3. Neelakanteshwara Temple, East Godavari\n4. Ramalingeswara Swamy Temple, Prakasam\n5. Kalahasteeswara Swamy Temple, Sri Kalahasthi'

In [222]:
answer_query_with_context("what are the durga temples in andhra pradesh?", df, document_embeddings)

('Andhra_Pradesh', 'Kanaka_Durga_Temple__Vijayawada__Andhra_Pradesh', 0)
('Andhra_Pradesh', 'Kanaka_Durga_Temple__Vijayawada__Andhra_Pradesh', 1)
('Odisha', 'Durga_Temple__Motia__Cuttack__Odisha', 0)
('Tamil_Nadu', 'Ujjaini_Mahakali_Temple__Secunderabad__Telangana', 2)
('Chhattisgarh', 'Hathnikala_Temple__Mungeli', 0)
('Delhi', 'Jhandewalan_Mandir__Delhi', 0)
('Maharashtra', 'Durga_Devi_temple__Ratnagiri', 0)
('Odisha', 'Vaishnodevi_Temple__Rourkela__Odisha', 1)
('Arunachal_Pradesh', 'Malinithan_temple_(or_Akashganga_temple)', 0)
('Madhya_Pradesh', 'Vindhyawasini_Mata_Temple__Sehore', 0)
('Madhya_Pradesh', 'Bijasan_Mata_Temple__Salkanpur', 0)
('Delhi', 'Chhatarpur_Temple__Mehrauli_Â\xa0New_Delhi', 1)
('Delhi', 'Jhandewalan_Temple__North', 1)
('Andhra_Pradesh', 'Kulandeshwara_Temple__Andhra_Pradesh', 0)
('Odisha', 'Parashurameshvara_Temple__Bhubaneswar__Odisha', 4)
('Andhra_Pradesh', 'Yaganti_temple__Andhra_Pradesh', 1)
('Chhattisgarh', 'Danteshwari_Temple__Danteshwari__Chhattisgarh', 1

C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1) Kanaka Durga Temple, Vijayawada \n2) Kulandeshwara Temple (exact location not provided) \n3) Yaganti Temple (Shiva, Parvati & Nandi are main deities at this temple, with Durga likely also worshipped.)'

In [223]:
answer_query_with_context("what are the major krishna temples in south india?", df, document_embeddings)

('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Kerala', 'Aranmula_Parthasarathy_Temple__Pathanamthitta__Kerala', 1)
('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Kerala', 'Thiruvanathu_Sree_Krishna_Temple_Thrissur_Kerala', 0)
('Tamil_Nadu', 'Aadhi_Kesava_Perumal_Temple__Vada_Madurai__Thiruvallur', 1)
('Tamil_Nadu', 'Uppiliappan_temple__Thanjavur__Tamil_Nadu', 2)
('Karnataka', 'Udupi_Sri_Krishna_Matha__Udupi__karnataka', 1)
('Andhra_Pradesh', 'Srikalahasti_Temple', 0)
('Tamil_Nadu', 'Sangameswarar_Temple__Erode__Tamil_Nadu', 1)
('Tamil_Nadu', 'Manalikarai_Azhvar_Krishna_Swamy_Temple__Kanyakumari', 2)
('Tamil_Nadu', 'Prasanna_Venkatesa_Perumal_Temple__Madurai', 1)
('Gujarat', 'Girnar__Junagadh__Gujarat', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'Some of the major Krishna temples in South India include Sri Pandurangan Temple in Thennangur, Aranmula Parthasarathy Temple in Pathanamthitta, Aadhi Kesava Perumal Temple in Vada Madurai, Uppiliappan Temple in Thanjavur, Thiruvanathu Sree Krishna Temple in Thrissur, and Prasanna Venkatesa Perumal Temple in Madurai.'

In [224]:
answer_query_with_context("what are the major venkateshwara temples in south india?", df, document_embeddings)

('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Tamil_Nadu', 'Sangameswarar_Temple__Erode__Tamil_Nadu', 1)
('Andhra_Pradesh', 'Venkateswara_Temple__Tirumala__Andhra_Pradesh', 0)
('Tamil_Nadu', 'Velladainatha_Swami_Temple__Thirukarugavur__Nagapattinam', 1)
('Tamil_Nadu', 'Kapaleeshwarar_Temple__Chennai__Tamil_Nadu', 1)
('Tamil_Nadu', 'Aadhi_Kesava_Perumal_Temple__Vada_Madurai__Thiruvallur', 1)
('Tamil_Nadu', 'Ramanathaswamy_Temple__Ramanathapuram__Tamil_Nadu', 4)
('Andhra_Pradesh', 'Kalahasteeswara_Swamy_Temple__Sri_Kalahasthi__Andhra_Pradesh', 0)
('Tamil_Nadu', 'Veerabhadra_Swamy_Temple__Mylapore__Chennai', 1)
('Andhra_Pradesh', 'Pallikondeswarar_Temple__Surutapalli__Chittoor', 0)
('Tamil_Nadu', 'Navaneetha_Krishnan_Temple__Arasalur__Trichy', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The Venkateswara Temple in Tirumala, Andhra Pradesh is one of the most prominent Vaishnavite temples in South India.'

In [225]:
answer_query_with_context("what are the major venkateshwara temples in north india?", df, document_embeddings)

('Tamil_Nadu', 'Chakravageswarar_Temple__Chakkarapalli__Thanjavur', 0)
('Gujarat', 'Girnar__Junagadh__Gujarat', 3)
('Tamil_Nadu', 'Sangameswarar_Temple__Erode__Tamil_Nadu', 1)
('Tamil_Nadu', 'Vaitheeswaran_Koil__Nagapattinam__Tamil_Nadu', 4)
('Tamil_Nadu', 'Aadhi_Kesava_Perumal_Temple__Vada_Madurai__Thiruvallur', 1)
('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Tamil_Nadu', 'Ramanathaswamy_Temple__Ramanathapuram__Tamil_Nadu', 4)
('Tamil_Nadu', 'Mahadevar_Temple__Edayarpakkam__Kanchipuram', 1)
('Kerala', 'Ananthapura_Lake_Temple__Kasaragod__Kerala', 2)
('Tamil_Nadu', 'Varadharaja_Perumal_Temple__Siruvapuri__Thiruvallur', 1)
('Uttarakhand', 'Guptakashi__Rudraprayag__Uttrakhand', 1)
('Uttarakhand', 'Madhyamaheshwar__Garhwal__Uttarakhand', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'There is no information related to Venkateshwara temples in North India in the given context.'

In [227]:
answer_query_with_context("what are the major krishna temples in north india?", df, document_embeddings)

('Gujarat', 'Girnar__Junagadh__Gujarat', 3)
('Delhi', 'Lakshmi_Narayan_Temple__Delhi', 0)
('Uttar_Pradesh', 'Krishna_Janmasthan_Temple_Complex__Mathura__Uttar_Pradesh', 0)
('Tamil_Nadu', 'Sri_Pandurangan_Temple__Thennangur', 2)
('Karnataka', 'Udupi_Sri_Krishna_Matha__Udupi__karnataka', 1)
('Odisha', 'Chateshwar_Temple__Cuttack__Odisha', 1)
('Haryana', 'Bhadrakali_Temple__Kurukshetra__Haryana', 1)
('Gujarat', 'Dwarkadhish_Temple__Dwarka__Gujarat', 1)
('Delhi', 'ISKCON_Temple_Delhi__New_Delhi', 0)
('Uttarakhand', 'Jyotirmath__Chamoli__Uttarakhand', 2)
('Haryana', 'Shri_Sthaneshwara_Temple__Kurukshetra__Haryana', 1)
('Rajasthan', 'Sanwariaji_Temple__Chittorgarh__Rajasthan', 0)
('West_Bengal', 'Dakshineswar_Kali_Temple__North_24_Parganas__West_Bengal', 1)
('Tamil_Nadu', 'Thiruneermalai_Dhivya_Desam__Kanchipuram', 3)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


"1. Krishna Janmasthan Temple Complex, Mathura, Uttar Pradesh\n2. Udupi Sri Krishna Matha, Udupi, Karnataka\n3. Sanwariaji Temple, Chittorgarh, Rajasthan\n4. Dakshineswar Kali Temple, North 24 Parganas, West Bengal \n5. Dwarkadhish Temple, Dwarka, Gujarat (Krishna's residential place)"

In [228]:
answer_query_with_context("what is the history of amarnath?", df, document_embeddings)

('Jammu_and_Kashmir', 'Shri_Amarnath_Shrine', 0)
('Jammu_and_Kashmir', 'Buddha_Amarnath_Temple__Poonch', 0)
('Maharashtra', 'Shiv_Mandir_Thane', 0)
('Madhya_Pradesh', 'Shyam_Kaka_Temple__Rajgarh__Madhya_Pradesh', 2)
('Tamil_Nadu', 'Kodi_Kuzhagar_Temple__Kodiakkarai__Nagapattinam', 1)
('Tamil_Nadu', 'Nandeeswarar_Temple__Nandivaram__Guduvanchery__Kanchipuram', 0)
('Karnataka', 'Amrutesvara_Temple__Amruthapura__Chikkamagaluru__Karnataka', 3)
('Uttar_Pradesh', 'Amareshwar_Temple__Varanasi', 0)
('Tamil_Nadu', 'Thirunandheeswarar_Temple__Manavur__Thiruvallur', 1)
('Tamil_Nadu', 'Kodi_Kuzhagar_Temple__Kodiakkarai__Nagapattinam', 2)
('Kerala', 'Thiruvanchikulam_Temple__Thrissur__kerala', 3)
('Tamil_Nadu', 'Aavundeeswarar_Temple__Nemam__Thiruvallur', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The holy cave or Amarnath Shrine is the abode of Lord Shiva. Shri Amarnath Shrine is located at the end of valley Lidder at a distance of 46 kms from Pahalgam and 14 kms from Baltal (Sonmarg), at an elevation of 3,888 meters. Amarnathji is considered to be one of the major Hindu Dhams. The holy ice-lingam is formed naturally in this cave. Darshan of holy lingam can be done in Month of Savan (August)through the Amarnath yatra which is organised by Amarnath shrine board. Commonly, pilgrimage journey can be opted by Chandanbari route via Pahalgam which is 96 kms from srinagar and 280 kms from Jammu. The world famous shrine of Amarnathji is situated about 48 Kms. from Pahalgam and attracts devotees from all over India.'

In [229]:
answer_query_with_context("how old is the amarnath shrine?", df, document_embeddings)

('Jammu_and_Kashmir', 'Shri_Amarnath_Shrine', 0)
('Jammu_and_Kashmir', 'Buddha_Amarnath_Temple__Poonch', 0)
('Maharashtra', 'Shiv_Mandir_Thane', 0)
('Karnataka', 'Shree_Vishnumurthy_Temple__Mangalore__karnataka', 1)
('Jammu_and_Kashmir', 'Sudh_Mahadev___Patnitop', 0)
('Tamil_Nadu', 'Vamanapureeswarar_Temple__Thirumanikuzhi__Cuddalore', 0)
('Uttar_Pradesh', 'Shivling_in_Devkali_Temple__Auraiya', 0)
('Uttar_Pradesh', 'Devi_Kali_Temple__Auraiya', 0)
('Jammu_and_Kashmir', 'Kameshwar_temple__Akhnoor', 0)
('Punjab', 'Mata_Chakreshwari_Temple__Fatehgarh_Sahib', 0)
('Jammu_and_Kashmir', 'Panchbakhtar_Temple__Jammu.', 0)
('Tamil_Nadu', 'Vamanapureeswarar_Temple__Thirumanikuzhi__Cuddalore', 4)
('Tamil_Nadu', 'Aavundeeswarar_Temple__Nemam__Thiruvallur', 0)
('Tamil_Nadu', 'Ramanatheeswarar_Temple__Porur__Chennai', 0)
('Tamil_Nadu', 'Edaganathar_Temple__Thiruvedagam__Madurai', 1)
('Madhya_Pradesh', 'Kalmadhav__Amarkantak', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_14152\1057417536.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The holy cave or Amarnath Shrine is believed to be one of the major Hindu Dhams and has a naturally formed ice-lingam. The shrine can be visited in the month of Savan (August) through the Amarnath yatra which is organized by Amarnath shrine board. The shrine attracts devotees from all over India and is located in the upper reaches of District Anantnag at an altitude of about 13,000 ft. from the sea level. Because of this height, the cave is covered with snow for most of the year. However, the context does not provide a specific age for the Amarnath shrine.'

In [26]:
answer_query_with_context("if i am in madurai and want to visit temples in a day, what can i visit?", df, document_embeddings)

('Tamil_Nadu', 'Kalamega_Perumal_Temple__Thirumohoor__Madurai', 0)
('Tamil_Nadu', 'Immayilum_Nanmai_Tharuvar_Temple__Madurai', 1)
('Tamil_Nadu', 'Meenakshi_Temple__Madurai__Tamil_Nadu', 4)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'You can visit Kalamega Perumal Temple in Thirumohoor and Immayilum Nanmai Tharuvar Temple in the heart of the city. Additionally, Meenakshi Temple is a prominent landmark in Madurai and attracts tens of thousands of visitors a day. All three temples can be visited in a day.'

In [25]:
answer_query_with_context("list vishnu temples near madurai", df, document_embeddings)

('Tamil_Nadu', 'Veerabhadra_Swamy_Temple__Mylapore__Chennai', 1)
('Tamil_Nadu', 'Thiruthevanartthogai_Madhava_Perumal_Temple__Thirunangur__Nagapattinam', 4)
('Tamil_Nadu', 'Arupadai_Veedu_Temple__Pazhamuthicholai__Madurai', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'1. Arupadai Veedu Temple - Pazhamuthicholai - Madurai (located on the northern outskirts of Madurai)\n2. Alagar Kovil Temple - Alagar Hills - Madurai (fortified temple complex revered as one of the 108 abodes of Vishnu glorified by the hymns of the Alwars)'

In [35]:
answer_query_with_context("what is the oldest temple in uttar pradesh?", df, document_embeddings)

('Bihar', 'Dekuli_Shiv_Mandir__Sheohar', 0)
('Gujarat', 'Girnar__Junagadh__Gujarat', 4)
('Uttar_Pradesh', 'Dwarkadheesh_Temple__Mathura__Uttar_Pradesh', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'The answer is Dwarkadheesh Temple, Mathura, Uttar Pradesh.'

In [34]:
answer_query_with_context("if i am in kanchipuram and want to visit temples in a day, what can i visit?", df, document_embeddings)

('Tamil_Nadu', 'Thiruvankaranai_Pattabhiramar_Temple__Kanchipuram', 0)
('Tamil_Nadu', 'Ona_Kantheeswarar_Temple__Panjupettai__Kanchipuram', 1)
('Kerala', 'Kadavallur_Sreeraman_Temple__Thrissur__Kerala', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'You can visit Thiruvankaranai Pattabhiramar Temple and Ona Kantheeswarar Temple in Kanchipuram.'

In [33]:
answer_query_with_context("if i am in hyderabad and want to visit temples in a day, what can i visit?", df, document_embeddings)

('Gujarat', 'The_Somnath_Temple__Gujarat', 1)
(nan, 'Hanuman_Temple__Â\xa0Hyderabad__Telangana', 0)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


KeyError: (nan, 'Hanuman_Temple__Â\xa0Hyderabad__Telangana', 0)

In [36]:
answer_query_with_context("if i am in varanasi and want to visit temples in a day, what can i visit?", df, document_embeddings)

('Uttar_Pradesh', 'Dhanvantareshwar_Temple__Varanasi', 4)
('Uttar_Pradesh', 'Sankat_Mochan_Hanuman_Temple__Varanasi__Uttar_Pradesh', 5)
('Uttar_Pradesh', 'Brihaspateeshwar_Temple__Varanasi', 1)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'You can visit the Dhanvantari Temple, the Sankat Mochan Hanuman Temple, and the Brihaspateeshwar Temple in Varanasi.'

In [44]:
answer_query_with_context("what are the most prominent temples in thiruvananthapuram?", df, document_embeddings)

('Kerala', 'Thiruvanchikulam_Temple__Thrissur__kerala', 2)
('Kerala', 'Peruvanam_Mahadeva_Temple__Thrissur__kerala', 0)
('Kerala', 'Thirunakkara_Sree_Mahadevar_Temple__Kottayam__kerala', 2)


C:\Users\003GHU744\AppData\Local\Temp\ipykernel_26804\1534434901.py:14: PerformanceWarning: indexing past lexsort depth may impact performance.
  document_section = df.loc[section_index].iloc[0]


'I cannot provide an answer as the context provided does not mention any temples in Thiruvananthapuram.'